In [1]:
AGENT_NAME = 'default_PPO_citylearn_challenge_2022_phase_2_Building_6_20_bins_500'
SAVE_DIR = 'attribution/'
ATTR = 'SVS 8000'

In [2]:
import pandas as pd
import numpy as np

import torchdrift.detectors as detectors
import torch

import KBMproject.utilities as utils

%matplotlib inline

In [3]:
baseline_SVS = np.loadtxt(f'baseline {ATTR}.csv',
                         delimiter=',',
                         ).astype('float32')
adv_acg_SVS = np.loadtxt(f'adv acg {ATTR}.csv',
                         delimiter=',',
                         ).astype('float32')
clean_acg_SVS = np.loadtxt(f'clean acg {ATTR}.csv',
                         delimiter=',',
                         ).astype('float32')
adv_bb_SVS = np.loadtxt(f'adv bb {ATTR}.csv',
                         delimiter=',',
                         ).astype('float32')
clean_bb_SVS = np.loadtxt(f'clean bb {ATTR}.csv',
                         delimiter=',',
                         ).astype('float32')
rebaseline_SVS = np.loadtxt(f'rebaseline {ATTR}.csv',
                         delimiter=',',
                         ).astype('float32')



In [4]:
BOOTSTRAP = 10_000
PVAL = 0.05
kernel = detectors.mmd.GaussianKernel()

In [5]:
result = detectors.kernel_mmd(torch.from_numpy(clean_bb_SVS).to('cuda'), 
                                  torch.from_numpy(adv_bb_SVS).to('cuda'), 
                                  n_perm=BOOTSTRAP,
                                  kernel=kernel)
print(f'The results for the clean and untargeted bb perturbed observations are mmd:{result[0]}, p-value:{result[1]}')

The results for the clean and untargeted bb perturbed observations are mmd:0.006551504135131836, p-value:0.0


In [6]:
result = detectors.kernel_mmd(torch.from_numpy(clean_acg_SVS).to('cuda'), 
                                  torch.from_numpy(adv_acg_SVS).to('cuda'), 
                                  n_perm=BOOTSTRAP,
                                  kernel=kernel)
print(f'The results for the clean and untargeted acg perturbed observations are mmd:{result[0]}, p-value:{result[1]}')

The results for the clean and untargeted acg perturbed observations are mmd:0.0019704103469848633, p-value:0.0


In [7]:
result = detectors.kernel_mmd(torch.from_numpy(clean_bb_SVS).to('cuda'), 
                                  torch.from_numpy(baseline_SVS).to('cuda'), 
                                  n_perm=BOOTSTRAP,
                                  kernel=kernel)
print(f'The results for the baseline unperturbed bb observations are mmd:{result[0]}, p-value:{result[1]}')

The results for the baseline unperturbed bb observations are mmd:0.0007040500640869141, p-value:9.999999747378752e-05


In [10]:
result = detectors.kernel_mmd(torch.from_numpy(clean_bb_SVS).to('cuda'), 
                                  torch.from_numpy(rebaseline_SVS).to('cuda'), 
                                  n_perm=BOOTSTRAP,
                                  kernel=kernel)
print(f'The results for the rebaseline unperturbed bb observations are mmd:{result[0]}, p-value:{result[1]}')

The results for the rebaseline unperturbed bb observations are mmd:0.0006994009017944336, p-value:9.999999747378752e-05


In [8]:
result = detectors.kernel_mmd(torch.from_numpy(clean_acg_SVS).to('cuda'), 
                                  torch.from_numpy(baseline_SVS).to('cuda'), 
                                  n_perm=BOOTSTRAP,
                                  kernel=kernel)
print(f'The results for the baseline unperturbed acg observations are mmd:{result[0]}, p-value:{result[1]}')

The results for the baseline unperturbed acg observations are mmd:0.0002529621124267578, p-value:0.22349999845027924


In [9]:
result = detectors.kernel_mmd(torch.from_numpy(clean_acg_SVS).to('cuda'), 
                                  torch.from_numpy(rebaseline_SVS).to('cuda'), 
                                  n_perm=BOOTSTRAP,
                                  kernel=kernel)
print(f'The results for the rebaseline unperturbed acg observations are mmd:{result[0]}, p-value:{result[1]}')

The results for the rebaseline unperturbed acg observations are mmd:0.000255584716796875, p-value:0.22299998998641968


In [11]:
result = detectors.kernel_mmd(torch.from_numpy(baseline_SVS).to('cuda'), 
                                  torch.from_numpy(rebaseline_SVS).to('cuda'), 
                                  n_perm=BOOTSTRAP,
                                  kernel=kernel)
print(f'The results for the rebaseline and baseline are mmd:{result[0]}, p-value:{result[1]}')

The results for the rebaseline and baseline are mmd:0.0001544952392578125, p-value:1.0


Overall we see that the MMD is an OoM begger between adversarial and clean distributions, though the pvals are all lower than 5%

Why are baseline and rebaseline so different? how was baselin generated? Were the actions recorded or predicted afterwards?